<h1 align="center">Elo-based Learner Model - SIDES </h1>
This is an implementation of Elo rating system. This mdoel estimates students' knowledge state on different specialties based on their performance on attempting questions in these specialty.

In [1]:
import os

import numpy as np
import pandas as pd

import datetime
import math
import time
import import_ipynb
import prepare_data_SIDES as sides_pdr
#import find_difficulty_with_irt as find_difficulty

from sklearn.metrics import accuracy_score

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
from sklearn.utils import resample

import dataio


from scipy import sparse

import json



importing Jupyter notebook from prepare_data_SIDES.ipynb


In [2]:
    """
In many educational applications there is also an asymmetry between correct and incorrect answers,
since students learn just by an exposure to an item. An answer to an item is not only evidence of student’s
knowledge, but also an opportunity for learning. In such situations we need to perform different updates
for correct and incorrect answers. Papouˇsek et al. (2014) and Pel´anek (2015) propose such modification of
the Elo rating system under the name “Performance Factor Analysis Extended / Elo” (PFAE)
    """
hyper_params = {
    'a_correct' : 1,
    'b_correct' : 0.5,
    'a_incorrect' : 1,
    'b_incorrect' : 0.5,
    'a_question' : 2.5, 
    'b_question' : 0.5,
    'a_specialty' : 2.5,
    'b_specialty' : 0.5
    }

##  Functions

#### Uncertainty Function : 
$$U(n) = \frac{a}{1 + b \times n}$$

- $ {a}$:constant hyper-parameters determining the starting value
- $ b$ :constant hyper-parameters determining the slope of changes
- n: number of prior updates on student’s knowledge state or item difficulty


An important difference between applications of the Elo rating system in games and education is an
asymmetry between students and items in education. For items we expect their difficulty to be approximately
constant. In some cases it may be useful to track changes in difficulty, e.g., in geography general knowledge
of some places may temporarily change due to their presence in media (Guinea during the Ebola epidemic).
But such cases are exceptions and changes in difficulty are not expected to be large. On the other hand,
changes in student skill are expected – after all, that is the aim of educational systems


In [3]:
def uncertainty_k(n, a, b,update_for):
    """
    - Most Elo extensions use an "uncertainty function" instead of a constant K.
    - The uncertainty function is used to adjust the size of the update based on the amount of data available.
    - For new players (students), the skill estimate is highly uncertain, and the update should be larger. As more data becomes available, the size of the update should get smaller.
    - Papoušek et al. (2014) and Nižnan et al. (2015) set `a` to 1 and `b` to 0.05 in their uncertainty function.
    - Abdi et al. (2019) set `a` to 1.8 and `b` to 0.05 in their experiments on public datasets.
    - In educational applications, there is often an asymmetry in the number of available answers for items and students. Each item is answered by many students, whereas for students, the number of answers is typically smaller by orders of magnitude.
    - Pelanek et al. (2016) suggest using different uncertainty functions for items and students may be useful in such cases.
    - Experience suggests that in the case of student modeling, it may be sufficient to model uncertainty in a simpler, pragmatic way (Nižnan et al., 2015).
    """
    u = a / (1 + (b*n))
    #u = 0.1
    # set a lower bound on uncertainty if update is for a student and if u<0.03
    if update_for == 'student' and np.any(u < 0.03):
        u = 0.03
    return u

#### Chance Level
If QUA (Question with Unique Answer) and question have ${n}_{opt}$ :

$$ P(\text{guessing}| {n}_{opt}, {type}_{answer})  = \frac{1} {n_{opt}} $$

If QMA (Question qith Multiple Answer) and question have ${n}_{opt}$ :

$$P(\text{guessing}| {n}_{opt}, {type}_{answer})  =  \frac{1} { \sum_{k=1}^{n_{opt}} ({n_{opt} \choose 1} , {n_{opt} \choose 2} , \cdots , {n_{opt} \choose n_{opt}})}$$


In [4]:
def probability_guessing(n_opt,answer_type):
    """
    Calculates the probability of a student answering a multiple-choice question
    (with n_opt options and n_correct_opt correct options) correctly by chance, assuming
    that the student can choose any number of options from 1 (see if you want 0 check option into probability?)
    to n_opt.
    Note that: If the question has only 1 correct answer then the probability is 1/n_opt. 
    (assuming that students know that they have to choose only 1 option) !!! Check if there is such info for multianswer
    """
    n_combinations = sum([math.comb(n_opt, k) for k in range(1,n_opt + 1)])
    p = 1 / (n_opt if answer_type == 'QUA' else n_combinations)
    
    
    return p

#### Probability that student un answers an item qm correctly : 
$$P(a_{nm}=1 | \bar{\lambda}_{nm}, d_m) = \sigma(\bar{\lambda}_{nm} - d_m)$$

- qm : question m tagged with g knowledge components including knowledge component $\delta_l$
- $\bar{\lambda}_{nl}$: un’s knowledge state on concept $\delta_l$
- $\bar{\lambda}_{nm} = \sum_{l=1}^L \lambda_{nl} \times \omega_{ml}$  :  un’s average competency on concepts that are associated with qm

#### If we add guessing to the probability of answering correctly :

$$P(\text{correct}_{nm} = 1)=P(\text{guessing}| {n}_{opt}, {type}_{answer}) + \frac{(1-P(\text{guessing}| {n}_{opt}, {type}_{answer}))}{1 + e^{-(\bar{\lambda}_{nm} - d_m)}} $$

In [5]:
def expect_score(student_rating, question_rating, n_options, answer_type):
    """
    Expected score of the interaction for a multiple-choice question with k options.
    """
    #guess_prob = chance_mat.loc[n_correct_options, n_options]
    guess_prob = probability_guessing(n_options,answer_type)  # probability of guessing the correct answer
    knowledge_prob = 1 - guess_prob  # probability of answering correctly based on knowledge
    logistic_input = 1 + np.exp(-(student_rating - question_rating))
    return guess_prob + (knowledge_prob / logistic_input)

In [6]:
def expect_score2(student_rating, question_rating, n_options, answer_type, specialty_rating):
    """
    Expected score of the interaction for a multiple-choice question with k options.
    """
    #guess_prob = chance_mat.loc[n_correct_options, n_options]
    guess_prob = probability_guessing(n_options,answer_type)  # probability of guessing the correct answer
    knowledge_prob = 1 - guess_prob  # probability of answering correctly based on knowledge
    logistic_input = 1 + np.exp(-(student_rating - (question_rating+specialty_rating)))
    return guess_prob + (knowledge_prob / logistic_input)

#### calculating RMSE: 

In [7]:
'''calculating RMSE (root mean squared error) based on model predictions and actual responses'''
def CalculateRMSE(Output, ground, I):
    Output = np.array(Output)
    ground = np.array(ground)
    error = (Output - ground) 
    err_sqr = error*error
    RMSE = math.sqrt(err_sqr.sum()/I)
    return RMSE  
def auc_roc(y, pred): 
    y = np.array(y)
    pred = np.array(pred)
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    auc = metrics.auc(fpr, tpr) 
    return auc

#### Run Elo 

##### Update Rules 
- For Question Difficulty: $$d_m := d_m +  U(n)(\mathrm{P}(a_{nm}=1|\bar{\lambda}{nm}, d_m) - a_{nm})$$
- For Student Ability in each tagged specialty : $$\lambda_{nl} := \lambda_{nl} +  U(n)(a_{nm} - \mathrm{P}(a_{nm}=1|\lambda_{nl}, d_m))$$

 where $ a_{nm}$ is the real (binary) result and $\alpha$ is a normalization factor, ensuring that the zero-sum game principles are enforced in the model

 $\alpha$ is computed using the following formula:
$$\alpha = \frac{|\mathrm{P}(a_{nm}=1|\bar{\lambda}{nm}, d{m})-a_{nm}|}{\sum\limits_{l=1}^{L}|\mathrm{a_{nm}}-\mathrm{P}(a_{nm}=1|\lambda_{nl},d_{m})\times\omega_{ml}|}$$

where $ \omega_{ml}$ is the number of specialties tagged by question m

In [8]:
def runelo_item_training(df,hyper_params=hyper_params,include_spec_difficulty=True):
    """
    Implementation of Elo rating system for adaptive educational learning platforms.
    This function is used on the training set to learn the difficulty of items. 
    Once the difficulty of items are learneres by this function, they are used in another training function (runelo_studentability_training)to learn the competency of students. 
    
    Arguments:
    df -- train data in the form of Pandas data frame
    
    Output: data_outputs that contains elo_ExpectedScore and actual_score for each line of df
    """
    # call hyper-parameters
    a_correct = hyper_params['a_correct']
    b_correct = hyper_params['b_correct']
    a_incorrect = hyper_params['a_incorrect']
    b_incorrect = hyper_params['b_incorrect']
    a_question = hyper_params['a_question']
    b_question = hyper_params['b_question']
    a_specialty = hyper_params['a_specialty']
    b_specialty = hyper_params['b_specialty']
    
    # Pre-compute values that depend only on the questions
    n_tagged_specialty_list = np.count_nonzero(q_mat, axis=1)
    wml_list = np.where(n_tagged_specialty_list > 0, 1 / n_tagged_specialty_list, 0)
    
    print("M-Elo execution for learning item difficulty is started.") 
    
    # Create arrays to store the expected score, actual score, student ID, and question ID for each row in df
    n_rows = len(df)
    elo_ExpectedScore = np.zeros(n_rows)
    actual = np.zeros(n_rows)
    student = np.zeros(n_rows, dtype=int)
    question=  np.zeros(n_rows, dtype=int)
    specialty = np.zeros(n_rows, dtype=object)
    question_dif = np.zeros(n_rows)
    student_average_ability = np.zeros(n_rows)
    specialty_average_difficulty= np.zeros(n_rows)
    
    # if learn_conf_interval:
    #     # Define the number of bootstrap samples
    #     num_bootstrap_samples = 1000
    #     # Add the 'upperbound' and 'lowerbound' columns with NaN values
    #     for item in question_difficulty_updates_list:
    #         item['upperbound'] = np.nan
    #         item['lowerbound'] = np.nan


    #start iteration over rows in df
    for count, (index, item) in enumerate(df.iterrows()):
        # Step 0: Initialization
        uid = item['user_id']
        qid = item['item_id']
        n_options = item['n_options']
        answer_type = item['answer_type']
        correct = item['correct']
        spec= item['kc_id']
        
        actual[count] = correct
        student[count] = uid
        question[count] = qid
        specialty[count] = spec

        wml = wml_list[qid] # weight for each specialty
        d = question_difficulty[qid] #getting the difficulty of the question qid from difficulty matrix
        c = learner_competency[uid] * q_mat[qid] # getting the competency of the student on the specialty associated with the question
        c_avg = np.sum(c)*wml # weighted average competency of the student on all the specialties associated with the question
        
        student_average_ability[count] = c_avg
        question_dif[count] = d
        
        topic_mask = q_mat[qid] != 0 ## Create a boolean mask indicating which elements in the q_mat row for the current question are non-zero

        
        ## Step 1: calculate the expected result from student and item point of view
        if include_spec_difficulty:
            d_spec= specialty_difficulty[0]*q_mat[qid]
            d_spec_avg = np.sum(d_spec)*wml # weighted average competency of the student on all the specialties associated with the question
            specialty_average_difficulty[count] = d_spec_avg
            # 1-a: calculate the expected outcome and alpha based on proficiency on each topic
            u_topic_expected_result = np.where(topic_mask, expect_score2(c, d, n_options, answer_type, d_spec), 0) # expected score is only calculated for topics where the corresponding entry in q_mat is non-zero
            # 1-b: calculate the expected outcome based on proficiency on all topics
            u_item_expected_result = expect_score2(c_avg, d, n_options,answer_type,d_spec_avg)
        else:
            # 1-a: calculate the expected outcome and alpha based on proficiency on each topic
            u_topic_expected_result = np.where(topic_mask, expect_score(c, d, n_options, answer_type), 0) # expected score is only calculated for topics where the corresponding entry in q_mat is non-zero
            # 1-b: calculate the expected outcome based on proficiency on all topics
            u_item_expected_result = expect_score(c_avg, d, n_options,answer_type)
            
        elo_ExpectedScore[count] = u_item_expected_result

        # Step 2: calculate normalization factor (alpha) to ensure that the zero-sum game principles are enforced in the model
        #alpha_topic = abs(correct - (u_topic_expected_result[topic_mask] * wml))
        #sum_alpha_topic = np.sum(alpha_topic)
        #if sum_alpha_topic == 0:
        #    sum_alpha_topic = 1e-10   
        #alpha = (abs(u_item_expected_result - correct)) / sum_alpha_topic

        # Step 3: updates
        # 3-a: update students ability on each specialty tagged by question
        # Vectorize the computation for each student and specialty
        n_prev_updates = attempt_counter_student_spec[uid, topic_mask]
        if correct == 1:
            change = uncertainty_k(n=n_prev_updates, a=a_correct, b=b_correct, update_for='student') * (correct - u_topic_expected_result[topic_mask])
        else:
            change = uncertainty_k(n=n_prev_updates,a=a_incorrect,b=b_incorrect, update_for='student') * (correct - u_topic_expected_result[topic_mask])
        learner_competency[uid, topic_mask] += change
        attempt_counter_student_spec[uid, topic_mask] += 1
        # Track student ability evolution
        mask_indices = np.where(topic_mask)[0]
        for spec_idx in mask_indices:
            student_ability_updates_list.append({'student': uid, 'specialty': spec_idx, 'ability': learner_competency[uid, spec_idx]})

        # # 3-b: update difficulty level of each questions
        n_prev_updates_q = attempt_counter_question[qid]
        change_question=uncertainty_k(n=n_prev_updates_q, a=a_question, b=b_question,update_for='question') * (u_item_expected_result - correct)
        question_difficulty[qid] += change_question
        #update attttempt_no
        attempt_counter_question[qid] += 1
        #track question difficulty evolution
        question_difficulty_updates_list.append({'question': qid, 'difficulty': question_difficulty[qid]})
        
        if include_spec_difficulty:
            # # 3-c: update difficulty level of each specialty
            n_prev_updates_spec = attempt_counter_spec[0, topic_mask]
            change_spec= uncertainty_k(n=n_prev_updates_spec, a=a_specialty, b=b_specialty, update_for='specialty') * (u_topic_expected_result[topic_mask]- correct)
            specialty_difficulty[0, topic_mask] += change_spec
            #update attttempt_no
            attempt_counter_spec[0, topic_mask] += 1
            # track specialty difficulty evolution
            for spec_idx in mask_indices:
                specialty_difficulty_updates_list.append({'specialty': spec_idx, 'difficulty': specialty_difficulty[0, spec_idx]})
            
        
        # if learn_conf_interval:
        #     # Step 4: bootstrapping
        #     # Convert the question_difficulty_updates_list to a DataFrame
        #     df_question_difficulty_updates = pd.DataFrame(question_difficulty_updates_list)
        #     # Filter the DataFrame based on the question ID
        #     question_difficulty_update_qid = df_question_difficulty_updates[df_question_difficulty_updates['question'] == qid]
        #     # Perform bootstrapping using the resample function from scikit-learn
        #     bootstrapped_difficulty = resample(question_difficulty_update_qid['difficulty'], replace=True, n_samples=num_bootstrap_samples)
        #     # Calculate the 95% confidence interval for the bootstrapped difficulty values
        #     lower_bound = np.percentile(bootstrapped_difficulty, 2.5)
        #     upper_bound = np.percentile(bootstrapped_difficulty, 97.5)
        #     # Update the upperbound and lowerbound columns in question_difficulty_updates_list but only for the last appended row
        #     # Update the 'lowerbound' and 'upperbound' values in the last appended row
        #     question_difficulty_updates_list[-1]['lowerbound'] = lower_bound
        #     question_difficulty_updates_list[-1]['upperbound'] = upper_bound

    print("M-Elo execution for learning item difficulty is ended.")
    # create a dictionary with column names as keys and lists as values
    data_outputs = {'student': student,'question': question, 'specialty': specialty, 'question_dif': question_dif, 'student_average_ability': student_average_ability, 'specialty_average_difficulty':specialty_average_difficulty, 'actual_score': actual, 'elo_ExpectedScore': elo_ExpectedScore }

    return data_outputs # later, delete returning values from this function



In [9]:
def runelo_studentability_training(df,hyper_params=hyper_params,include_spec_difficulty=True):

    """
    Implementation of Elo rating system for adaptive educational learning platforms.
    This function is used to update the competency of students based on the difficulty of items obtained
    from runelo_item_training function. 
    !! Please be advised that in this function, the difficulty of items do not change and only the competency of students are changing. 
    
    Arguments:
    df -- train data in the form of Pandas data frame
    
    Output:
    """
    # call hyper-parameters
    a_correct = hyper_params['a_correct']
    b_correct = hyper_params['b_correct']
    a_incorrect = hyper_params['a_incorrect']
    b_incorrect = hyper_params['b_incorrect']
    
    # Pre-compute values that depend only on the questions
    n_tagged_specialty_list = np.count_nonzero(q_mat, axis=1)
    wml_list = np.where(n_tagged_specialty_list > 0, 1 / n_tagged_specialty_list, 0)
    
    print("M-Elo execution for estimating student competency is started.") 
    
    # Create arrays to store the expected score, actual score, student ID, and question ID for each row in df
    n_rows = len(df)
    elo_ExpectedScore = np.zeros(n_rows)
    actual = np.zeros(n_rows)
    student = np.zeros(n_rows, dtype=int)
    question=  np.zeros(n_rows, dtype=int)
    specialty = np.zeros(n_rows, dtype=object)
    question_dif = np.zeros(n_rows)
    student_average_ability = np.zeros(n_rows)
    specialty_average_difficulty= np.zeros(n_rows)
    
    for count, (index, item) in enumerate(df.iterrows()):
        # Step 0: Initialization
        uid = item['user_id']
        qid = item['item_id']
        n_options = item['n_options']
        answer_type = item['answer_type']
        correct = item['correct']
        spec= item['kc_id']

        
        actual[count] = correct
        student[count] = uid
        question[count] = qid
        specialty[count] = spec
        
        wml = wml_list[qid]
        d = question_difficulty[qid] #getting the difficulty of the question qid from difficulty matrix
        c = learner_competency[uid] * q_mat[qid] # getting the competency of the student on the topics associated with the question
        c_avg = np.sum(c)*wml # weighted average competency of the student on the topics associated with the question

        student_average_ability[count] = c_avg
        question_dif[count] = d
        
        topic_mask = q_mat[qid] != 0 ## Create a boolean mask indicating which elements in the q_mat row for the current question are non-zero
        
        ## Step 1: calculate the expected result from student and item point of view
        if include_spec_difficulty:
            d_spec= specialty_difficulty[0]*q_mat[qid]
            d_spec_avg = np.sum(d_spec)*wml # weighted average competency of the student on all the specialties associated with the question
            specialty_average_difficulty[count] = d_spec_avg
            # 1-a: calculate the expected outcome and alpha based on proficiency on each topic
            u_topic_expected_result = np.where(topic_mask, expect_score2(c, d, n_options, answer_type, d_spec), 0) # expected score is only calculated for topics where the corresponding entry in q_mat is non-zero
            # 1-b: calculate the expected outcome based on proficiency on all topics
            u_item_expected_result = expect_score2(c_avg, d, n_options,answer_type,d_spec_avg)
        else:
            # 1-a: calculate the expected outcome and alpha based on proficiency on each topic
            u_topic_expected_result = np.where(topic_mask, expect_score(c, d, n_options, answer_type), 0) # expected score is only calculated for topics where the corresponding entry in q_mat is non-zero
            # 1-b: calculate the expected outcome based on proficiency on all topics
            u_item_expected_result = expect_score(c_avg, d, n_options,answer_type)
            
        elo_ExpectedScore[count] = u_item_expected_result

        # Step 2: calculate normalization factor (alpha) to ensure that the zero-sum game principles are enforced in the model
        #alpha_topic = abs(correct - (u_topic_expected_result[topic_mask] * wml))
        #sum_alpha_topic = np.sum(alpha_topic)
        #if sum_alpha_topic == 0:
        #    sum_alpha_topic = 1e-10  
        #alpha = abs(u_item_expected_result - correct) / sum_alpha_topic

        # Step 3: updates. here no need to update the difficulty of the question and specialty
        # 3-a: update students ability on each specialty tagged by question
        # Vectorize the computation for each student and specialty
        n_prev_updates = attempt_counter_student_spec[uid, topic_mask]
        if correct == 1:
            change = uncertainty_k(n=n_prev_updates, a=a_correct, b=b_correct,update_for='student') * (correct - u_topic_expected_result[topic_mask])
        else:
            change = uncertainty_k(n=n_prev_updates,a=a_incorrect,b=b_incorrect,update_for='student') * (correct - u_topic_expected_result[topic_mask])
            
        learner_competency[uid, topic_mask] += change
        attempt_counter_student_spec[uid, topic_mask] += 1

        # Track student ability evolution
        mask_indices = np.where(topic_mask)[0]
        for spec_idx in mask_indices:
            student_ability_updates_list.append({'student': uid, 'specialty': spec_idx, 'ability': learner_competency[uid, spec_idx]})
    
    print("M-Elo execution for estimating student competency is ended.")
    # create a dictionary with column names as keys and lists as values
    data_outputs = {'student': student,'question': question, 'specialty': specialty, 'question_dif': question_dif, 'student_average_ability': student_average_ability, 'specialty_average_difficulty':specialty_average_difficulty, 'actual_score': actual, 'elo_ExpectedScore': elo_ExpectedScore }
    
    return data_outputs  


### Funtion for mapping the question difficulty found by logreg to the elo difficulty

In [10]:
def get_question_difficulty_form_previous_year(question_difficulty_irt, old_new_item_ids_irt, old_new_item_ids_elo, question_difficulty,question_difficulty_updates_list,attempt_counter_question_irt,attempt_counter_question, count_attemp=True, reverse_coef=True):
        
        # multiply difficulty_irt column with -1 if reverse_coef is True
        if reverse_coef:
                question_difficulty_irt['difficulty_irt'] = question_difficulty_irt['difficulty_irt'] * -1
        # fill the question_difficulty with the question_difficulty found by irt
        # merge the two dataframes on the column 'item_id'
        question_difficulty_irt = pd.merge(question_difficulty_irt, old_new_item_ids_irt, on='item_id')
        # remove the column 'item_id'
        question_difficulty_irt = question_difficulty_irt.drop(columns=['item_id'])
        # merge the two dataframes on the 'question' column
        merged = pd.merge(old_new_item_ids_elo, question_difficulty_irt, on='question', how='left')
        #remove the column 'question'
        merged = merged.drop(columns=['question'])
        # remove the rows that have nan values in the column 'difficulty_irt'
        merged = merged.dropna(subset=['difficulty_irt'])
        # fill the question_difficulty array with the item difficulties found by irt
        item_numbers= merged['item_id'].values
        difficulties = merged['difficulty_irt'].values
        question_difficulty[item_numbers] = difficulties
        
        
        # fill the question_difficulty_updates_list with the item difficulties found by irt
        # create a dictionary with item_id as key and irt difficulty as value
        item_difficulty_dict = dict(zip(merged['item_id'], merged['difficulty_irt']))
        # replace the 0 in question_difficulty_updates_list with the corresponding difficulty_irt
        for item in question_difficulty_updates_list:
                if item['question'] in item_difficulty_dict:
                        item['difficulty'] = item_difficulty_dict[item['question']]
                        
        
        if count_attemp:
                # fill the attempt_counter_question with the attempt_counter found by irt
                # merge attempt_counter_question_irt with attempt_counter_question
                attempt_counter_question_irt = pd.merge(attempt_counter_question_irt, old_new_item_ids_irt, on='item_id')
                # remove the column 'item_id'
                attempt_counter_question_irt = attempt_counter_question_irt.drop(columns=['item_id'])
                # merge old_new_item_ids_elo with attempt_counter_question_irt
                merged = pd.merge(old_new_item_ids_elo, attempt_counter_question_irt, on='question', how='left')
                # remove the column 'question'
                merged = merged.drop(columns=['question'])
                # remove the rows that have nan values in the column 'attempt_counter'
                merged = merged.dropna(subset=['count'])
                # fill the attempt_counter_question array with the attempt_counter found by irt
                item_numbers= merged['item_id'].values
                attempt_counter = merged['count'].values
                attempt_counter_question[item_numbers] = attempt_counter
                  
        return question_difficulty, question_difficulty_updates_list, attempt_counter_question

### Funtion for mapping the student ability found by logreg to the elo student ability

In [11]:
def find_ability(merged_dict, user_id, specialty):
    user_ids = merged_dict['user_id']
    specialties = merged_dict['specialty']
    abilities = merged_dict['ability']

    if user_id in user_ids and specialty in specialties:
        user_index = user_ids.index(user_id)
        specialty_index = specialties.index(specialty)
        ability = abilities[user_index][specialty_index]
        return ability
    else:
        return None

In [12]:

def get_user_ability_form_previous_year(stundet_ability_irt, old_new_user_ids_irt, old_new_user_ids_elo, learner_competency,student_ability_updates_list, attempt_counter_student_spec_irt, attempt_counter_student_spec,old_new_skill,count_attemp=True, reverse_coef=False):
        
        # multiply the columns from 1 to end of the stundet_ability_irt by -1  if reverse_coef is True
        if reverse_coef:
                stundet_ability_irt.iloc[:, 1:] = stundet_ability_irt.iloc[:, 1:] * -1
                
        # merge the two dataframes on the column 'item_id'
        stundet_ability_irt = pd.merge(stundet_ability_irt, old_new_user_ids_irt, on='user_id')        # remove the column 'item_id'
        stundet_ability_irt = stundet_ability_irt.drop(columns=['user_id'])
        
        # merge the two dataframes on the 'question' column
        merged = pd.merge(old_new_user_ids_elo, stundet_ability_irt, on='student', how='left')
        #remove the column 'question'
        merged = merged.drop(columns=['student'])
        
        # remove the rows that have nan values in any column 
        merged = merged.dropna()
        
        # turn old_new_skill into an array
        # Convert the array to a DataFrame
        old_new_skill = pd.DataFrame(old_new_skill)
        # Rename the column as 'skill_name'
        old_new_skill = old_new_skill.rename(columns={0: 'skill_name'})

        # Reset the index and rename the index column
        old_new_skill = old_new_skill.reset_index().rename(columns={'index': 'skill_id'})
        
        # Create a dictionary mapping skill_names to skill_ids
        skill_mapping = old_new_skill.set_index('skill_name')['skill_id'].to_dict()

        # create a dictionary to map the column names of merged to the corresponding skill_id
        dict_user={'used_id':'user_id'}
        # join a and skill_mapping
        dict_user.update(skill_mapping)
        
        merged = merged.rename(columns=skill_mapping)
        
        # turn merged unto dictionary
        merged_dict = {
        'user_id': merged['user_id'].tolist(),
        'specialty': merged.columns[1:].tolist(),
        'ability': merged.iloc[:, 1:].values.tolist()}
        
        # fill the learner_competency
        for i in range(len(merged_dict['user_id'])):
                for j in range(len(merged_dict['specialty'])):
                        learner_competency[merged_dict['user_id'][i]][merged_dict['specialty'][j]] = merged_dict['ability'][i][j]
                        
        # Update 'ability' values in student_ability_updates_list
        for update in student_ability_updates_list:
                if update['student'] in merged_dict['user_id'] and update['specialty'] in merged_dict['specialty']:
                        student = update['student']
                        specialty = update['specialty']
                        ability=find_ability(merged_dict, student, specialty)
                        update['ability'] = ability
        
        if count_attemp:
                # Update 'attempt' values in attempt_counter_student_spec
                attempt_counter_student_spec_irt = pd.merge(attempt_counter_student_spec_irt, old_new_user_ids_irt, on='user_id')
                # remove the column 'item_id'
                attempt_counter_student_spec_irt = attempt_counter_student_spec_irt.drop(columns=['user_id'])
                # merge old_new_item_ids_elo with attempt_counter_question_irt
                merged = pd.merge(old_new_user_ids_elo, attempt_counter_student_spec_irt, on='student', how='left')
                # remove the column 'question'
                merged = merged.drop(columns=['student'])
                # remove the rows that have nan values in the column 'attempt_counter'
                merged = merged.dropna()
                
        
        merged = merged.rename(columns=skill_mapping)
        
        # turn merged unto dictionary
        merged_dict = {
        'user_id': merged['user_id'].tolist(),
        'specialty': merged.columns[1:].tolist(),
        'attempt': merged.iloc[:, 1:].values.tolist()}
        
        # fill the learner_competency
        for i in range(len(merged_dict['user_id'])):
                for j in range(len(merged_dict['specialty'])):
                        attempt_counter_student_spec[merged_dict['user_id'][i]][merged_dict['specialty'][j]] = merged_dict['attempt'][i][j]
                        

        return learner_competency, student_ability_updates_list, attempt_counter_student_spec

### Funtion for mapping the specialty difficulty found by logreg to the elo specialty difficulty

In [13]:
def get_spec_difficulty_form_previous_year(specialty_difficulty_irt, old_new_skill, attempt_counter_spec_irt, specialty_difficulty, specialty_difficulty_updates_list, attempt_counter_spec, count_attemp=True, reverse_coef=True):

    if reverse_coef:
        specialty_difficulty_irt['specialty_difficulty'] = specialty_difficulty_irt['specialty_difficulty'] * -1

    # fill the specialty_difficulty with the specialty_difficulty found by irt
    # turn old_new_skill into an array
    old_new_skill_df = pd.DataFrame(old_new_skill)
    # rename the column of old_new_skill as 'specialty'
    old_new_skill_df = old_new_skill_df.rename(columns={0: 'specialty'})
    # rename the index of old_new_skill as 'specialty_id'
    old_new_skill_df = old_new_skill_df.reset_index().rename(columns={'index': 'specialty_id'})
    # merge the two dataframes on the 'specialty' column
    merged = pd.merge(old_new_skill_df, specialty_difficulty_irt, on='specialty', how='left')
    # also merge attempt_counter_specialty_irt with merged on the 'specialty' column
    merged = pd.merge(merged, attempt_counter_spec_irt, on='specialty', how='left')
    # remove nan values
    merged = merged.dropna(subset=['specialty_difficulty'])
    # put the values of 'specialty_difficulty' column to the corresponding row in specialty_difficulty
    specialty_numbers= merged['specialty_id'].values
    difficulties = merged['specialty_difficulty'].values
    specialty_difficulty[specialty_numbers] = difficulties

    # fill the specialty_difficulty_updates_list with the specialty_difficulty found by irt
    if count_attemp:
        # put the values of 'attempt_counter' column to the corresponding row in attempt_counter_spec
        attempt_counter_spec[specialty_numbers] = merged['total_attempts'].values
        
    # fill the specialty_difficulty_updates_list with the specialty_difficulty found by irt
    # create a dictionary with specialty_id as key and irt difficulty as value
    specialty_difficulty_dict = dict(zip(merged['specialty_id'], merged['specialty_difficulty']))
    # replace the 0 in specialty_difficulty_updates_list with the corresponding difficulty_irt
    for item in specialty_difficulty_updates_list:
            if item['specialty'] in specialty_difficulty_dict:
                    item['difficulty'] = specialty_difficulty_dict[item['specialty']]
                    
    return specialty_difficulty, specialty_difficulty_updates_list, attempt_counter_spec
    
            

## Run Model on Data ##

### Define HyperParameters

###### to be able to learn from data, it needs to have enough data available. As our analysis and previous experience shows, the system needs at least 100 students to get good estimates of item difficulty (Pelanek,2016)

In [14]:
configurations = {
    '2020-2021' : {
    'folder' : 'data/sides',
    'course' : '2020-2021',
    'min_interactions_per_user' : 100,
    'min_answer_per_question' : 100,
    'kc_column' : 'specialty', 
    'train_file' : 'train_data.csv', 
    'test_file' : 'test_data.csv',
    'initial_points_question_logreg': True,
    'initial_points_students_logreg': True,
    'include_spec_difficulty': True,
    'initial_points_specdifficulty_logreg': True,
    'logreg_year': '2019-2020',
    'logreg_model': 'user_skill_together-spec_difficulty-items-weighted_encoding',
    'already_preprocessed': True,
    'first_go': True,
    'second_go': True
    }
}

**DETERMINE THE COURSE ID IN THE FOLLOWING CELL**

In [15]:
# dettermine course_id
course_id = '2020-2021'

In [16]:
folder = configurations[course_id]['folder']
course = configurations[course_id]['course']
min_interactions_per_user = configurations[course_id]['min_interactions_per_user']
min_answer_per_question = configurations[course_id]['min_answer_per_question']
kc_column = configurations[course_id]['kc_column']
train_file = configurations[course_id]['train_file']
test_file = configurations[course_id]['test_file']
initial_points_question_logreg=configurations[course_id]['initial_points_question_logreg']
initial_points_students_logreg=configurations[course_id]['initial_points_students_logreg']
include_spec_difficulty=configurations[course_id]['include_spec_difficulty']
initial_points_specdifficulty_logreg=configurations[course_id]['initial_points_specdifficulty_logreg']
already_preprocessed=configurations[course_id]['already_preprocessed']


#### Save results seperately for seperate versions of the model & save features

In [17]:
all_features = []

if configurations[course_id]['include_spec_difficulty']:
    all_features.append('specdif_logreg' if configurations[course_id]['initial_points_specdifficulty_logreg'] else 'specdif_0')

all_features.append('q_logreg' if configurations[course_id]['initial_points_question_logreg'] else 'q_0')
all_features.append('skill_logreg' if configurations[course_id]['initial_points_students_logreg'] else 'skill_0')

all_features.extend(['1st', '2nd'] if configurations[course_id]['first_go'] and configurations[course_id]['second_go'] else [])

additional_suffix = '-'.join(all_features)

EXPERIMENT_FOLDER = f"{folder}/{course_id}/result_elo_{additional_suffix}/"

dataio.prepare_folder(EXPERIMENT_FOLDER)

# Save the configurations and hyper_params to a TXT file

# Combine configurations and hyper_params into a single dictionary
combined_dict = {
    'configurations': configurations[course_id],
    'hyper_params': hyper_params
}

# Write the configurations dictionary to the TXT file
with open(EXPERIMENT_FOLDER + 'features.txt', 'w') as file:
    json.dump(combined_dict, file, indent=4)

print("Features have been written to the file: features.txt in the ", EXPERIMENT_FOLDER)

Features have been written to the file: features.txt in the  data/sides/2020-2021/result_elo_specdif_logreg-q_logreg-skill_logreg-1st-2nd/


#### Preprocessing the Data

###### If data already preprocessed call directly (uncomment the second cell)

In [18]:
if already_preprocessed and os.path.exists(folder+'/'+ course +"/processed/preprocessed_data.csv"):
    # print message
    print("Data already preprocessed. Reading preprocessed data...")
    # read preprocessed data
    pre_processed_data = pd.read_csv(folder+'/'+ course +"/processed/preprocessed_data.csv")
    # read q-matrix
    q_mat =q_mat = sparse.load_npz(folder + '/' + course + "/processed/q_mat.npz").toarray()
    # read listOfKC.json
    with open(folder + '/' + course + "/processed/listOfKC.json") as json_file:
        listOfKC = json.load(json_file)
    with open(folder + '/' + course + "/processed/dict_of_kc.json") as json_file:
        dict_of_kc = json.load(json_file)
        
    # read train_set
    train_set = pd.read_csv(folder + '/' + course + "/processed/train_set.csv")
    # read test_set
    test_set = pd.read_csv(folder + '/' + course + "/processed/test_set.csv")
    # read skill_names_ids_map_df
    skill_names_ids_map_df = pd.read_csv(folder + '/' + course + "/processed/skill_names_ids_map.csv")

else: # if data is not preprocessed
    # print message
    print("Data not preprocessed. Preprocessing data...")
    # import warnings
    warnings.filterwarnings(action='once')
    # processing the row data made available by KDD organisers
    pre_processed_data, q_mat, listOfKC, dict_of_kc, train_set, test_set,skill_names_ids_map_df = sides_pdr.prepare_sides(folder, course, \
                                                                       train_file, \
                                                                       test_file,\
                                                                        kc_column, min_interactions_per_user, min_answer_per_question,\
                                                                        True, True, True,True)


Data already preprocessed. Reading preprocessed data...


In [19]:
# print("Reading Train and Test sets.")
# train_set = pd.read_csv(folder + '/'+course+"/processed/train_set.csv")
# test_set = pd.read_csv(folder + '/'+course+"/processed/test_set.csv")
# pre_processed_data = pd.read_csv(folder + '/'+course+"/processed/preprocessed_data.csv")

In [20]:
print("Shape of train set is:", train_set.shape)
print("Shape of test set is:", test_set.shape)

Shape of train set is: (17904661, 8)
Shape of test set is: (4387302, 8)


In [21]:
# number of students
uSize = pre_processed_data['user_id'].nunique()
print("Number of students is:", uSize)

qSize = pre_processed_data['item_id'].nunique()
print("Number of questions is:", qSize)

tSize = len(listOfKC)
print("Number of specialty is:", tSize)

Number of students is: 8590
Number of questions is: 74703
Number of specialty is: 31


### Define chance_mat (guessing probability matrix) ###

In [22]:
# first create a new dataframe with only the unique combinations of 'n_opt' and 'n_correct_options'
unique_combinations = pre_processed_data[['n_options', 'answer_type']].drop_duplicates().reset_index(drop=True)

# Apply probability_guessing function to unique combinations
unique_combinations['probability_guessing'] = unique_combinations.apply(
    lambda row: probability_guessing(int(row['n_options']), row['answer_type']), axis=1)

chance_mat = pd.pivot_table(unique_combinations, values='probability_guessing', index=['answer_type'], columns=['n_options'])


In [23]:
question_difficulty = np.zeros(qSize) #stores difficulty level of each question after each answer
learner_competency = np.zeros((uSize, tSize)) #stores student's profiency level on each topic and is updated upon each attempt
unique_questions = pre_processed_data['item_id'].unique()
question_difficulty_updates_list = [{'question': qid, 'difficulty': 0} for qid in unique_questions]
attempt_counter_question = np.zeros(qSize)
attempt_counter_student_spec=np.zeros((uSize, tSize))

'''
For students, we create a dataframe, in which index column corresponds
to student_id and columns correspond to the topic of the course. It is 
first populated by all zeros and then it is updated accordingly. 
If the attempt_counter_student_spec is zero, it shows that it is the first time it has
been attempted. Otherwise, the attempt_counter_student_spec should be updated accordingly.
'''

unique_students = pre_processed_data['user_id'].unique()
specialties = list(dict_of_kc.values())
student_ability_updates_list = [{'student': uid, 'specialty': spec, 'ability': 0} for uid in unique_students for spec in specialties]

# learn question difficulty and student ability from irt if initial_points_irt is True
if initial_points_question_logreg: # if initial_points_irt is True (this is defined in the configurations dictionary)
    print("Learning question difficulty from LogReg.")
    # print that its needed to learn question difficulty from irt by runnin irt_for_large_data.ipynb for the irt_year defined in the configurations dictionary
    if not os.path.exists(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+ "/question_difficulty.csv"):
        print("Please run loistic regression for the year "+configurations[course_id]['logreg_year']+" to learn question difficulty from IRT. and rerun this cell.")
    #    # learn question difficulty from irt by calling the difficulties_irt function from find_difficulty_with_irt.py
    #    question_difficulty = find_difficulty.difficulties_irt(train_set, q_mat, question_difficulty, StudPerQuest=100, QuestPerStud=100, HighQuestNb=True)
    #    # save the question difficulty in a csv file
    #    np.savetxt(folder + '/'+course+"/processed/question_difficulty_irt.csv", question_difficulty, delimiter=",")
    
    else:
        print("Reading question difficulty from file.")
        # read item_deltas.csv file
        question_difficulty_irt = pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+ "/question_difficulty.csv")
        old_new_item_ids_irt= pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/processed"+"/old_new_item_ids.csv")
        old_new_item_ids_elo = pd.read_csv(folder + '/'+course+"/processed/old_new_item_ids.csv") 
        attempt_counter_question_irt= pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/processed"+"/attempt_counter_question.csv")
        # get question difficulty and question_difficulty_updates_list from get_question_difficulty_form_previous_year function
        question_difficulty, question_difficulty_updates_list , attempt_counter_question = get_question_difficulty_form_previous_year(question_difficulty_irt, old_new_item_ids_irt, old_new_item_ids_elo, question_difficulty, question_difficulty_updates_list,attempt_counter_question_irt,attempt_counter_question,count_attemp=False, reverse_coef=True)
else:
    print("Learning question difficulty from ELO directly.")

# learn question difficulty and student ability from irt if initial_points_irt is True
if initial_points_students_logreg: # if initial_points_irt is True (this is defined in the configurations dictionary)
    print("Learning student ability from LogReg.")
    # print that its needed to learn question difficulty from irt by runnin irt_for_large_data.ipynb for the irt_year defined in the configurations dictionary
    if not os.path.exists(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+"/learner_competency.csv"):
        print("Please run logistic regressin for the year "+configurations[course_id]['logreg_year']+" to learn user ability from IRT. and rerun this cell.")

    else:
        if not os.path.exists(folder + '/'+ configurations[course_id]['logreg_year']+"/processed/attempt_counter_student_spec.csv"):
            print("Please run logistic reression for the year "+configurations[course_id]['logreg_year']+" to learn attempt_counter_student_spec from IRT. and rerun this cell.")
        else:
            print("Reading question difficulty from file.")
            # read item_deltas.csv file
            stundet_ability_irt = pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+"/learner_competency.csv")
            old_new_user_ids_irt= pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/processed"+"/old_new_user_ids.csv")
            attempt_counter_student_spec_irt= pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/processed"+"/attempt_counter_student_spec.csv")
            old_new_user_ids_elo = pd.read_csv(folder + '/'+course+"/processed/old_new_user_ids.csv")
            old_new_skill= listOfKC.copy()
            # get question difficulty and question_difficulty_updates_list from get_question_difficulty_form_previous_year function
            learner_competency, student_ability_updates_list, attempt_counter_student_spec = get_user_ability_form_previous_year(stundet_ability_irt, old_new_user_ids_irt, old_new_user_ids_elo, learner_competency,student_ability_updates_list, attempt_counter_student_spec_irt, attempt_counter_student_spec,old_new_skill,count_attemp=False, reverse_coef=False)
else:
    print("Learning student ability from ELO directly.")
    
if include_spec_difficulty:
    print("specialty difficulty is included.")
    # open a specialty_difficulty array with specialty names from listOfKC in the first column and zeros in the second column
    specialty_difficulty = np.zeros((len(specialties)))
    specialty_difficulty_updates_list = [{'specialty': spec, 'difficulty': 0} for spec in specialties]
    attempt_counter_spec=np.zeros((len(specialties)))
    
    # learn specialty difficulty from irt if initial_points_specdifficulty_logreg is True
    if initial_points_specdifficulty_logreg:
        print("Learning specialty difficulty from LogReg.")
        # print that its needed to learn question difficulty from irt by runnin irt_for_large_data.ipynb for the irt_year defined in the configurations dictionary
        if not os.path.exists(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+"/specialty_difficulty.csv"):
            print("Please run irt_for_large_data.ipynb for the year "+configurations[course_id]['logreg_year']+" to learn specialty difficulty from IRT. and rerun this cell.")
            
        else:
            if not os.path.exists(folder + '/'+ configurations[course_id]['logreg_year']+"/processed/attempt_counter_spec.csv"):
                print("Please run irt_for_large_data.ipynb for the year "+configurations[course_id]['logreg_year']+" to learn attempt_counter_spec from IRT. and rerun this cell.")
                
            else:
                print("Reading specialty difficulty from file.")
                # read specialty_difficulty.csv file
                specialty_difficulty_irt = pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/result_logreg_" + configurations[course_id]['logreg_model']+"/specialty_difficulty.csv")
                attempt_counter_spec_irt= pd.read_csv(folder + '/'+ configurations[course_id]['logreg_year']+"/processed"+"/attempt_counter_spec.csv")
                old_new_skill= listOfKC.copy()
                # get specialty difficulty and specialty_difficulty_updates_list from get_question_difficulty_form_previous_year function
                specialty_difficulty, specialty_difficulty_updates_list, attempt_counter_spec = get_spec_difficulty_form_previous_year(specialty_difficulty_irt, old_new_skill, attempt_counter_spec_irt, specialty_difficulty, specialty_difficulty_updates_list, attempt_counter_spec, count_attemp=False, reverse_coef=True)
    else:
        print("Learning specialty difficulty from ELO directly.")
        

    # reshape the specialty_diffciulty and attempt_counter_spec arrays
    specialty_difficulty = specialty_difficulty.reshape((1,len(specialties)))
    attempt_counter_spec = attempt_counter_spec.reshape((1,len(specialties)))
    
# number of times a question on the defined topic is solved
#attempt_counter_student_spec = pd.DataFrame(0, index=np.arange(uSize), columns=list(dict_of_kc.values()))

#number of time a question was answered by each user
#attempt_counter_question = pd.DataFrame(0, index=np.arange(qSize), columns=["n_attempt"])



Learning question difficulty from LogReg.
Reading question difficulty from file.
Learning student ability from LogReg.
Reading question difficulty from file.
specialty difficulty is included.
Learning specialty difficulty from LogReg.
Reading specialty difficulty from file.


###### Training for learning item difficulty


In [24]:
train_set.sort_values(by=["timestamp", "item_id"], inplace=True) #first, timestamp should be converted to datetime
train_set.reset_index(inplace=True, drop=True)

In [25]:
# ### Calling Elo function for train data to learn the difficulty of items ###
## when doing training for learning item difficulty, don't forget to first sort values by their timestamp. 
warnings.filterwarnings(action='once') 
if configurations[course_id]['first_go']:
    data_outputs = runelo_item_training(train_set,include_spec_difficulty=configurations[course_id]['include_spec_difficulty'])

M-Elo execution for learning item difficulty is started.
M-Elo execution for learning item difficulty is ended.


In [26]:

if not os.path.isdir(EXPERIMENT_FOLDER+"/training_output_1go"):
    os.makedirs(EXPERIMENT_FOLDER+"/training_output_1go")
student_ability_updates=pd.DataFrame(student_ability_updates_list)
question_difficulty_updates=pd.DataFrame(question_difficulty_updates_list)
student_ability_updates.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/student_ability_updates.csv", index=True)
question_difficulty_updates.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/question_difficulty_updates.csv", index=True)
question_difficulty_df = pd.DataFrame(question_difficulty)
learner_competency_df = pd.DataFrame(learner_competency)
question_difficulty_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/question_difficulty.csv", index=True)
learner_competency_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/learner_competency.csv", index=True)
attempt_counter_question_df = pd.DataFrame(attempt_counter_question)
attempt_counter_question_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/attempt_counter_question.csv", index=True)
attempt_counter_student_spec_df = pd.DataFrame(attempt_counter_student_spec)
attempt_counter_student_spec_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/attempt_counter_student_spec.csv", index=True)

# create a DataFrame from the dictionary
data_outputs = pd.DataFrame(data_outputs)
data_outputs.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/data_outputs.csv", index=True)


if include_spec_difficulty:
    specialty_difficulty_updates_df=pd.DataFrame(specialty_difficulty_updates_list)
    specialty_difficulty_updates_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/specialty_difficulty_updates.csv", index=True)
    specialty_difficulty_df = pd.DataFrame(specialty_difficulty)
    specialty_difficulty_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/specialty_difficulty.csv", index=True)
    attempt_counter_spec_df = pd.DataFrame(attempt_counter_spec)
    attempt_counter_spec_df.to_csv(EXPERIMENT_FOLDER+"/training_output_1go/attempt_counter_spec.csv", index=True)


###### Training for learning user competency

In [27]:
warnings.filterwarnings(action='once')
if configurations[course_id]['second_go']:
    ### Calling Elo function for train data to learn competency of students ###
    ## Re-initializing students parameters
    learner_competency = np.zeros((uSize, tSize)) #stores student's profiency level on each topic and is updated upon each attempt
    attempt_counter_student_spec=np.zeros((uSize, tSize))

    unique_students = pre_processed_data['user_id'].unique()
    specialties = list(dict_of_kc.values())
    student_ability_updates_list = [{'student': uid, 'specialty': spec, 'ability': 0} for uid in unique_students for spec in specialties]

    data_outputs = runelo_studentability_training(train_set,include_spec_difficulty=configurations[course_id]['include_spec_difficulty'])

M-Elo execution for estimating student competency is started.
M-Elo execution for estimating student competency is ended.


###### Save update tracking for training

In [28]:
# Save data
if not os.path.isdir(EXPERIMENT_FOLDER+"/training_output"):
    os.makedirs(EXPERIMENT_FOLDER+"/training_output")
student_ability_updates=pd.DataFrame(student_ability_updates_list)
question_difficulty_updates=pd.DataFrame(question_difficulty_updates_list)
student_ability_updates.to_csv(EXPERIMENT_FOLDER+"/training_output/student_ability_updates.csv", index=True)
question_difficulty_updates.to_csv(EXPERIMENT_FOLDER+"/training_output/question_difficulty_updates.csv", index=True)
question_difficulty_df = pd.DataFrame(question_difficulty)
learner_competency_df = pd.DataFrame(learner_competency)
question_difficulty_df.to_csv(EXPERIMENT_FOLDER+"/training_output/question_difficulty.csv", index=True)
learner_competency_df.to_csv(EXPERIMENT_FOLDER+"/training_output/learner_competency.csv", index=True)
attempt_counter_student_spec_df = pd.DataFrame(attempt_counter_student_spec)
attempt_counter_student_spec_df.to_csv(EXPERIMENT_FOLDER+"/training_output/attempt_counter_student_spec.csv", index=True)
attempt_counter_question_df = pd.DataFrame(attempt_counter_question)
attempt_counter_question_df.to_csv(EXPERIMENT_FOLDER+"/training_output/attempt_counter_question.csv", index=True)

# create a DataFrame from the dictionary
data_outputs = pd.DataFrame(data_outputs)
data_outputs.to_csv(EXPERIMENT_FOLDER+"/training_output/data_outputs.csv", index=True)

df = pd.DataFrame(listOfKC)
df.to_csv(EXPERIMENT_FOLDER+"/training_output/listOfKC.csv", index=True)

if include_spec_difficulty:
    specialty_difficulty_updates_df=pd.DataFrame(specialty_difficulty_updates_list)
    specialty_difficulty_updates_df.to_csv(EXPERIMENT_FOLDER+"/training_output/specialty_difficulty_updates.csv", index=True)
    specialty_difficulty_df = pd.DataFrame(specialty_difficulty)
    specialty_difficulty_df.to_csv(EXPERIMENT_FOLDER+"/training_output/specialty_difficulty.csv", index=True)
    attempt_counter_spec_df = pd.DataFrame(attempt_counter_spec)
    attempt_counter_spec_df.to_csv(EXPERIMENT_FOLDER+"/training_output/attempt_counter_spec.csv", index=True)


###### Test for learning item difficulty 


In [29]:
#create temporary copie to be used in learning user rating (not affected by item training)
learner_competency_tmp = learner_competency.copy()
attempt_counter_student_spec_tmp = attempt_counter_student_spec.copy()
student_ability_updates_tmp = student_ability_updates_list.copy()

In [30]:
# ### Calling Elo function for train data to learn the difficulty of items ###
## when doing training for learning item difficulty, don't forget to first sort values by their timestamp. 
test_set.sort_values(by=["timestamp", "item_id"], inplace=True) #first, timestamp should be converted to datetime
test_set.reset_index(inplace=True, drop=True)

In [31]:
warnings.filterwarnings(action='once')
if configurations[course_id]['first_go']:
        data_outputs = runelo_item_training(test_set,include_spec_difficulty=configurations[course_id]['include_spec_difficulty'])

M-Elo execution for learning item difficulty is started.
M-Elo execution for learning item difficulty is ended.


###### Test for learning user rating


In [32]:
warnings.filterwarnings(action='once')
if configurations[course_id]['second_go']:
    learner_competency = learner_competency_tmp.copy()
    attempt_counter_student_spec = attempt_counter_student_spec_tmp.copy()
    student_ability_updates_list = student_ability_updates_tmp.copy()
    data_outputs = runelo_studentability_training(test_set,include_spec_difficulty=configurations[course_id]['include_spec_difficulty'])

M-Elo execution for estimating student competency is started.
M-Elo execution for estimating student competency is ended.


In [33]:
# Save data and listOfKC

# create output directory if it doesn't exist
if not os.path.isdir(EXPERIMENT_FOLDER+"/test_output"):
    os.makedirs(EXPERIMENT_FOLDER+"/test_output")
data_outputs = pd.DataFrame(data_outputs)
data_outputs.to_csv(EXPERIMENT_FOLDER+"/test_output/data_outputs.csv", index=True)
question_difficulty_df = pd.DataFrame(question_difficulty)
question_difficulty_df.to_csv(EXPERIMENT_FOLDER+"/test_output/question_difficulty.csv", index=True)


###### AUC ACC and RMSE results for model

In [34]:
rmse_test = CalculateRMSE(data_outputs["elo_ExpectedScore"], data_outputs["actual_score"], len(data_outputs["elo_ExpectedScore"]))
auc_test = auc_roc(data_outputs["actual_score"], data_outputs["elo_ExpectedScore"])
acc_test = accuracy_score(np.array(data_outputs["actual_score"]), np.array(data_outputs["elo_ExpectedScore"]).round(), normalize=True)
print("Test RMSE: ", rmse_test)
print("Test AUC: ", auc_test)
print("Test ACC: ", acc_test)

# save the resut as txt file
with open(os.path.join(EXPERIMENT_FOLDER, "validation_results.txt"), "w") as f:
    f.write("Test RMSE: " + str(rmse_test) + "\n")
    f.write("Test AUC: " + str(auc_test) + "\n")
    f.write("Test ACC: " + str(acc_test) + "\n")
    


Test RMSE:  0.42011140529065305
Test AUC:  0.8115394070176233
Test ACC:  0.7326947176191655
